In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import preprocess as pre
import time

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

In [3]:
# --- Data Preprocessing --- 
start = time.time()
pre.ais_trainCleanup("../CSV/big_files/ais_train.csv", "ais_train_processed.csv")
end = time.time()

print(end-start)

,time|cog|sog|rot|heading|navstat|etaRaw|latitude|longitude|vesselId|portId
0,2024-01-01 00:00:25|284.0|0.7|0|88|0|01-09 23:...
1,2024-01-01 00:00:36|109.6|0.0|-6|347|1|12-29 2...
2,2024-01-01 00:01:45|111.0|11.0|0|112|0|01-02 0...
3,2024-01-01 00:03:11|96.4|0.0|0|142|1|12-31 20:...
4,2024-01-01 00:03:51|214.0|19.7|0|215|0|01-25 1...
